In [5]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [6]:
Rebuild_data=True

In [9]:
class DogsVsCats:
    img_size=50
    cats='/kaggle/input/dogs-cats-images/dog vs cat/dataset/training_set/cats/'
    dogs='/kaggle/input/dogs-cats-images/dog vs cat/dataset/training_set/dogs/'
    labels={cats:0,dogs:1}
    training_data=[]
    catcount=0
    dogcount=0
                
    def make_training_data(self):
        for label in self.labels:
            try:
                for f in tqdm(os.listdir(label)):
                    path=os.path.join(label,f)
                    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                    img=cv2.resize(img,(self.img_size,self.img_size))
                    self.training_data.append([np.array(img),np.eye(2)[self.labels[label]]])
                    
                    if label == self.cats:
                        self.catcount+=1
                    elif label == self.dogs:
                        self.dogcount+=1
            except Exception as e:
                pass  
            
        np.random.shuffle(self.training_data)
        np.save('training_data.npy',self.training_data)
        print('Cats:',self.catcount)
        print('Dogs:',self.dogcount)

In [10]:
if Rebuild_data:
    dogsvcats = DogsVsCats()
    dogsvcats.make_training_data()

In [11]:
training_data=np.load('training_data.npy',allow_pickle=True)

In [12]:
training_data

In [13]:
import matplotlib.pyplot as plt

plt.imshow(training_data[0][0])
plt.show()

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [54]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,32,5)     #f=5 - that is the frame window will be of size 5
        self.conv2=nn.Conv2d(32,64,5)
        self.conv3=nn.Conv2d(64,128,5)
        
        x=torch.randn(50,50).view(-1,1,50,50)
        self._to_linear=None
        self.convs(x)
        
        self.fc1=nn.Linear(self._to_linear,512)
        self.fc2=nn.Linear(512,2)

    def convs(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        
        #print(x[0].shape)
        if self._to_linear is None:
            self._to_linear=x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self,x):
        x=self.convs(x)
        x=x.view(-1,self._to_linear)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.softmax(x,dim=1)

In [56]:
net=Net()

In [57]:
net

In [58]:
#Splitting training data into X and y values and creating a ratio to split the data into train and test
import torch.optim as optim

optimizer=optim.Adam(net.parameters(),lr=0.001)
loss_function=nn.MSELoss()

X=torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X=X/255.0
y=torch.Tensor([i[1] for i in training_data])

val_pct=0.1
val_size=int(len(X)*val_pct)
print(val_size)

In [59]:
#Splitting the data into train and valid set
train_X=X[:-val_size]
train_y=y[:-val_size]

val_x=X[-val_size:]
val_y=y[-val_size:]
print(val_y.shape)
print(train_y.shape)

# Why do we need to mark the gradient zero while training pytorch model

In PyTorch, for every mini-batch during the training phase, we typically want to explicitly set the gradients to zero before starting to do backpropragation (i.e., updating the Weights and biases) because PyTorch accumulates the gradients on subsequent backward passes. This accumulating behaviour is convenient while training RNNs or when we want to compute the gradient of the loss summed over multiple mini-batches. So, the default action has been set to accumulate (i.e. sum) the gradients on every loss.backward() call.

Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly. Otherwise, the gradient would be a combination of the old gradient, which you have already used to update your model parameters, and the newly-computed gradient.

In [60]:
#training the model and accounting the loss 
batch_size=100
epoch=1

for epoch in range(epoch):
    for i in tqdm(range(0,len(train_X),batch_size)):
        #print(i,i+batch_size)
        batch_X=train_X[i:i+batch_size].view(-1,1,50,50)
        batch_y=train_y[i:i+batch_size]
        
        net.zero_grad()    #if we are combining multiple network then we might use optimizer.zero_grad as there will be mulitple optimizer
        outputs=net(batch_X)
        loss=loss_function(outputs,batch_y)
        loss.backward()
        optimizer.step()

print(loss)

In [62]:
#checking the accuracy of the model on validation set
correct=0
total=0

with torch.no_grad():
    for i in tqdm(range(len(val_x))):
        real_class=torch.argmax(val_y[i])
        net_out=net(val_x[i].view(-1,1,50,50))[0]
        predicted_class=torch.argmax(net_out)
        if real_class == predicted_class:
            correct+=1
        total+=1

print('Accuracy:',round(correct/total,3))

In [65]:
#To validate if you are accessing GPU
torch.cuda.is_available()

In [66]:
#Assigning device
device=torch.device('cuda:0')
print(device)

In [67]:
if torch.cuda.is_available():
    device=torch.device('cuda:0')
    print('Running on GPU')
else:
    device=torch.device('cpu')
    print('Running on CPU')

In [68]:
torch.cuda.device_count()

In case we had multiple GPU then we can distribute the load among the GPUs

In [69]:
net.to(device)

In [82]:
#Train the model on GPU
def train(net):
    
    epoch=10
    optimizer=optim.Adam(net.parameters(),lr=0.001)
    loss_function=nn.MSELoss()
    
    for epoch in range(epoch):
        for i in tqdm(range(0,len(train_X),batch_size)):
            #print(i,i+batch_size)
            batch_X=train_X[i:i+batch_size].view(-1,1,50,50).to(device)
            batch_y=train_y[i:i+batch_size].to(device)
            
            net.zero_grad()    #if we are combining multiple network then we might use optimizer.zero_grad as there will be mulitple optimizer
            outputs=net(batch_X)
            loss=loss_function(outputs,batch_y)
            loss.backward()
            optimizer.step()
        print(f'Epoch:{epoch},loss: {loss}')

In [83]:
train(net)

In [84]:
#Run the test set on GPU
def test(net):
    
    correct=0
    total=0
    
    with torch.no_grad():
        for i in tqdm(range(len(val_x))):
            real_class=torch.argmax(val_y[i]).to(device)
            net_out=net(val_x[i].view(-1,1,50,50).to(device))[0]
            predicted_class=torch.argmax(net_out)
            if real_class == predicted_class:
                correct+=1
            total+=1
    print('Accuracy:',round(correct/total,3))

In [85]:
test(net)